### Introduction

Data wrangling is a core skill that everyone who works with data should be familiar with since so much of the world's data isn't clean. The repurcussions of ignoring data wrangling before analysis can be so bad and sometimes unimaginable. If we analyze, visualize, or model our data before we wrangle it, our consequences could be making mistakes, missing out on cool insights, and wasting time. So best practices say wrangle. Always.

In this project, we gather data from various sources and in variety of forms. When data is sourced from different sources, there are extreme inconsistencies between them. So inorder to analyse this data, cleaning must be considered first.

The dataset that we will be wrangling (and analyzing and visualizing) is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs.

In [88]:
# import libraries

import pandas as pd
import requests
import time
import json
import tweepy
import os

### Gather Data

#### Gather data 1 - from the downloadable link

In [89]:
# load tweets archieve data
tweets_df = pd.read_csv('twitter-archive-enhanced.csv')
tweets_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


#### Gather data 2 - fetch data from the url using requests library

In [90]:
# get image preictions data

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open(os.path.join('./', url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

In [91]:
# load image predictions data

images_df = pd.read_csv('image-predictions.tsv', sep='\t')
images_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


#### Gather data 3 - More info from WeRateDogs using Twitter API

In [92]:
# twitter api tokens

consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [93]:
# deleted_ids = []
# with open(os.path.join('./', 'tweet_json.txt'), mode='w') as file:
#     for id in tweets_df.tweet_id:
#         start = time.time()
#         try:
#             data = api.get_status(id, tweet_mode='extended', wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
#             file.write(json.dumps(data._json) + '\n')
#         except:
#             deleted_ids.append(id)
#         end = time.time()
#         print(str(data.id) + "------------" + str(end-start))
# len(deleted_ids)

In [94]:
# read tweet_json.txt and create an appropriate dataframe
df_list = []
with open('tweet_json.txt', 'r') as file:
    lines = file.read().splitlines()
    for line_data in lines:
        json_data = json.loads(line_data)
        tweet_id = json_data['id']
        favorite_count = json_data['favorite_count']
        retweet_count = json_data['retweet_count']
        df_list.append({
            'tweet_id': tweet_id,
            'favorite_count': favorite_count,
            'retweet_count': retweet_count
        })
likes_df = pd.DataFrame(df_list)
likes_df.head()

,favorite_count,retweet_count,tweet_id
0,38388,8431,892420643555336193
1,32894,6211,892177421306343426
2,24786,4111,891815181378084864
3,41723,8560,891689557279858688
4,39897,9285,891327558926688256


### Assess Data 

#### Visual Assessment 

In [95]:
tweets_df

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [96]:
images_df

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [97]:
likes_df

,favorite_count,retweet_count,tweet_id
0,38388,8431,892420643555336193
1,32894,6211,892177421306343426
2,24786,4111,891815181378084864
3,41723,8560,891689557279858688
4,39897,9285,891327558926688256
5,20022,3081,891087950875897856
6,11722,2045,890971913173991426
7,64797,18697,890729181411237888
8,27538,4222,890609185150312448
9,31599,7321,890240255349198849


#### Programmatic Assessment

In [98]:
likes_df.shape, images_df.shape, tweets_df.shape

((2341, 3), (2075, 12), (2356, 17))

In [99]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [100]:
images_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [101]:
likes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341 entries, 0 to 2340
Data columns (total 3 columns):
favorite_count    2341 non-null int64
retweet_count     2341 non-null int64
tweet_id          2341 non-null int64
dtypes: int64(3)
memory usage: 54.9 KB


In [102]:
# checks for retweets
tweets_df.query('retweeted_status_id in tweet_id')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lilly. She just paralle...,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Emmy. She was adopted t...,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Shadow. In an attempt to r...,8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
74,878316110768087041,NaN,NaN,2017-06-23 18:17:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Terrance. He's being yelle...,6.690004e+17,4.196984e+09,2015-11-24 03:51:38 +0000,https://twitter.com/dog_rates/status/669000397...,11,10,Terrance,None,None,None,None
91,874434818259525634,NaN,NaN,2017-06-13 01:14:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Coco. At first I though...,8.663350e+17,4.196984e+09,2017-05-21 16:48:45 +0000,https://twitter.com/dog_rates/status/866334964...,12,10,Coco,None,None,None,None
95,873697596434513921,NaN,NaN,2017-06-11 00:25:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Walter. He won't start ...,8.688804e+17,4.196984e+09,2017-05-28 17:23:24 +0000,https://twitter.com/dog_rates/status/868880397...,14,10,Walter,None,None,None,None
97,873337748698140672,NaN,NaN,2017-06-10 00:35:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Sierra. She's one preci...,8.732138e+17,4.196984e+09,2017-06-09 16:22:42 +0000,https://www.gofundme.com/help-my-baby-sierra-g...,12,10,Sierra,None,None,pupper,None
109,871166179821445120,NaN,NaN,2017-06-04 00:46:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Dawn. She's just checki...,8.410770e+17,4.196984e+09,2017-03-13 00:02:39 +0000,https://twitter.com/dog_rates/status/841077006...,12,10,Dawn,None,None,None,None
118,869988702071779329,NaN,NaN,2017-05-31 18:47:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: We only rate dogs. This is quit...,8.591970e+17,4.196984e+09,2017-05-02 00:04:57 +0000,https://twitter.com/dog_rates/status/859196978...,12,10,quite,None,None,None,None


In [103]:
sum(tweets_df.expanded_urls.isnull())

59

In [104]:
tweets_df.rating_numerator.unique()

array([  13,   12,   14,    5,   17,   11,   10,  420,  666,    6,   15,
        182,  960,    0,   75,    7,   84,    9,   24,    8,    1,   27,
          3,    4,  165, 1776,  204,   50,   99,   80,   45,   60,   44,
        143,  121,   20,   26,    2,  144,   88])

In [105]:
tweets_df.rating_denominator.unique()

array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
       130, 110,  16, 120,   2])

### Issues List

#### Quality Issues:
- retweets are included.
- Names of the dogs are wrong (a, an, the, this, etc)
- retweeted_status_timestamp and timestamp are not datetime objects and in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id should be str object
- null represented as 'None' in columns 'name', 'doggo', 'floofer', 'pupper','puppo' instead of NaN.
- Ratings are not extracted properly from text column.
- tweet_id - 835246439529840640 has a 0 denominator rating.
- missing tweet images(expanded url).
- number of observations(rows) is not consistent - (images_df: 2075 instead of tweets_df: 2356)

#### Tidiness Issues:
- stage (doggo, floofer, pupper, puppo) in 4 different columns
- All data is not present in one dataset

### Cleaning data:

In [106]:
# copy all dataframes to another variable

tweet_clean = tweets_df.copy()
images_clean = images_df.copy()
likes_clean = likes_df.copy()

#### Define
- Drop the retweets data

#### Clean

In [107]:
# query for retweets

retweets = tweet_clean.query('retweeted_status_id in tweet_id')

In [108]:
# drop the retweets columns

tweet_clean.drop(retweets.index, inplace=True)

#### Test

In [109]:
retweets = tweet_clean.query('retweeted_status_id in tweet_id')
retweets

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


#### Define
 - Replace the inappropriate names of the dogs to None

#### Clean

In [110]:
# replace all faulty names to None

tweet_clean['name'].replace('the', 'None', inplace=True)
tweet_clean['name'].replace("light",'None', inplace=True)
tweet_clean['name'].replace("life",'None', inplace=True)
tweet_clean['name'].replace("an",'None', inplace=True)
tweet_clean['name'].replace("a",'None', inplace=True)
tweet_clean['name'].replace("his","None", inplace=True)
tweet_clean['name'].replace("by",'None', inplace=True)
tweet_clean['name'].replace("actually",'None', inplace=True)
tweet_clean['name'].replace("just",'None', inplace=True)
tweet_clean['name'].replace("getting",'None', inplace=True) 
tweet_clean['name'].replace("infuriating",'None', inplace=True) 
tweet_clean['name'].replace("old",'None', inplace=True) 
tweet_clean['name'].replace("all",'None', inplace=True) 
tweet_clean['name'].replace("this",'None', inplace=True) 
tweet_clean['name'].replace("very",'None', inplace=True) 
tweet_clean['name'].replace("not",'None', inplace=True)
tweet_clean['name'].replace("one",'None', inplace=True)
tweet_clean['name'].replace("my",'None', inplace=True)
tweet_clean['name'].replace("O","None", inplace=True)
tweet_clean['name'].replace("quite","None", inplace=True)
tweet_clean['name'].replace("such","None", inplace=True)

#### Test

In [111]:
tweet_clean.name.value_counts()

None        804
Charlie      11
Lucy         11
Oliver       10
Cooper       10
Penny         9
Tucker        9
Sadie         8
Lola          8
Winston       8
Daisy         7
Toby          7
Oscar         6
Jax           6
Bella         6
Bo            6
Bailey        6
Koda          6
Stanley       6
Buddy         5
Louis         5
Bentley       5
Dave          5
Leo           5
Milo          5
Chester       5
Rusty         5
Scout         5
Bear          4
Clarence      4
           ... 
Monty         1
Shawwn        1
Filup         1
Mason         1
Chubbs        1
Shaggy        1
Aiden         1
Trip          1
Blanket       1
Rumpole       1
Pablo         1
Peanut        1
Freddery      1
Bowie         1
Ralph         1
Amélie        1
Jeffri        1
Marlee        1
Dixie         1
Astrid        1
Benny         1
Tito          1
Zoe           1
Miguel        1
Pumpkin       1
Vince         1
Lilli         1
Julio         1
Major         1
Simba         1
Name: name, Length: 936,

#### Define
- Change retweeted_status_timestamp and timestamp to datetime objects and convert in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id to str objects

#### Clean

In [112]:
# correcting the tweets_clean datatypes

tweet_clean.timestamp = pd.to_datetime(tweet_clean.timestamp)
tweet_clean.retweeted_status_timestamp = pd.to_datetime(tweet_clean.retweeted_status_timestamp)
tweet_clean.in_reply_to_status_id = tweet_clean.in_reply_to_status_id.astype(str)
tweet_clean.in_reply_to_user_id = tweet_clean.in_reply_to_user_id.astype(str)
tweet_clean.retweeted_status_id = tweet_clean.retweeted_status_id.astype(str)
tweet_clean.retweeted_status_user_id = tweet_clean.retweeted_status_user_id.astype(str)

#### Test

In [113]:
tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2200 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2200 non-null int64
in_reply_to_status_id         2200 non-null object
in_reply_to_user_id           2200 non-null object
timestamp                     2200 non-null datetime64[ns]
source                        2200 non-null object
text                          2200 non-null object
retweeted_status_id           2200 non-null object
retweeted_status_user_id      2200 non-null object
retweeted_status_timestamp    25 non-null datetime64[ns]
expanded_urls                 2141 non-null object
rating_numerator              2200 non-null int64
rating_denominator            2200 non-null int64
name                          2200 non-null object
doggo                         2200 non-null object
floofer                       2200 non-null object
pupper                        2200 non-null object
puppo                         2200 non-null object
dtypes: datetim

#### Define
- Remove tweets which has 0 denominator rating

#### Clean

In [114]:
drop_data = tweet_clean.query('rating_denominator <= 0')
tweet_clean.drop(drop_data.index, inplace=True)

#### Test

In [116]:
tweet_clean.rating_denominator.unique()

array([ 10,  15,  70,   7, 150,  11, 170,  20,  50,  90,  80,  40, 130,
       110,  16, 120,   2])

#### Define
- Remove tweets with missing tweet images.

#### Clean

In [117]:
tweet_clean = tweet_clean[tweet_clean.expanded_urls.notnull()]

#### Test

In [118]:
sum(tweet_clean.expanded_urls.isnull())

0

#### Define
- Make number of observations(rows) consistent across the tables

#### Clean

In [119]:
images_clean.shape, tweet_clean.shape, likes_clean.shape

((2075, 12), (2141, 17), (2341, 3))

In [120]:
# remove the rows whose data is deleted from WeRateDogs account
likes_tweet_ids = likes_clean.tweet_id
results = tweet_clean.query('tweet_id not in @likes_tweet_ids')
tweet_clean.drop(results.index, inplace=True)

In [121]:
images_clean.shape, tweet_clean.shape, likes_clean.shape

((2075, 12), (2137, 17), (2341, 3))

In [122]:
# retrive the tweet_ids that are present in tweets and images data
images_tweet_ids = images_clean.tweet_id
tweet_ids = tweet_clean.tweet_id
common_ids = set(tweet_ids) & set(images_tweet_ids)

In [123]:
# drop the rows whose image info is not provided
results = tweet_clean.query('tweet_id not in @common_ids')
tweet_clean.drop(results.index, inplace=True)
tweet_clean.shape

(2006, 17)

In [124]:
# drop the rows whose tweets data is not present
results = images_clean.query('tweet_id not in @common_ids')
images_clean.drop(results.index, inplace=True)
images_clean.shape

(2006, 12)

In [125]:
# make likes_df consitent to others
results = likes_clean.query('tweet_id not in @common_ids')
likes_clean.drop(results.index, inplace=True)
likes_clean.shape

(2006, 3)

#### Test

In [126]:
images_clean.shape, tweet_clean.shape, likes_clean.shape

((2006, 12), (2006, 17), (2006, 3))

#### Define
- Change None to NaN in columns 'name', 'doggo', 'floofer', 'pupper','puppo'

#### Clean

In [127]:
tweet_clean.name.replace('None', 'NaN', inplace=True)
tweet_clean.doggo.replace('None', 'NaN', inplace=True)
tweet_clean.floofer.replace('None', 'NaN', inplace=True)
tweet_clean.pupper.replace('None', 'NaN', inplace=True)
tweet_clean.puppo.replace('None', 'NaN', inplace=True)

#### Test

In [128]:
tweet_clean.name.unique()

array(['Phineas', 'Tilly', 'Archie', 'Darla', 'Franklin', 'NaN', 'Jax',
       'Zoey', 'Cassie', 'Koda', 'Bruno', 'Ted', 'Stuart', 'Oliver', 'Jim',
       'Zeke', 'Ralphus', 'Gerald', 'Jeffrey', 'Canela', 'Maya', 'Mingus',
       'Derek', 'Roscoe', 'Waffles', 'Jimbo', 'Maisey', 'Earl', 'Lola',
       'Kevin', 'Yogi', 'Noah', 'Bella', 'Grizzwald', 'Rusty', 'Gus',
       'Stanley', 'Alfy', 'Koko', 'Rey', 'Gary', 'Elliot', 'Louis',
       'Jesse', 'Romeo', 'Bailey', 'Duddles', 'Jack', 'Steven', 'Beau',
       'Snoopy', 'Shadow', 'Emmy', 'Aja', 'Penny', 'Dante', 'Nelly',
       'Ginger', 'Benedict', 'Venti', 'Goose', 'Nugget', 'Cash', 'Jed',
       'Sebastian', 'Sierra', 'Monkey', 'Harry', 'Kody', 'Lassie', 'Rover',
       'Napolean', 'Boomer', 'Cody', 'Rumble', 'Clifford', 'Dewey',
       'Scout', 'Gizmo', 'Walter', 'Cooper', 'Harold', 'Shikha', 'Lili',
       'Jamesy', 'Coco', 'Sammy', 'Meatball', 'Paisley', 'Albus',
       'Neptune', 'Belle', 'Quinn', 'Zooey', 'Dave', 'Jersey', 'Hobbes'

#### Define
- Change the rating_numerator and rating_denominator columns to float and assign the correct float values

#### Clean

In [129]:
# check if there are float values in text column
tweet_clean[tweet_clean.text.str.contains(r'(\d+(\.\d+))\/(\d+)')]['text'].values

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


array([ 'This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948',
       "This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",
       "This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq",
       'Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD'], dtype=object)

In [130]:
# extract ratings from text column
ratings = tweet_clean['text'].str.extract(('(\d+(\.\d+)|(\d+))\/(\d+)'), expand=False)

In [131]:
# name ratings column
ratings.columns = ['num_float', 'float_decimal', 'num_interger', 'den_interger']

In [132]:
# check if correct float values are extracted from text column
ratings[ratings.num_float!=ratings.num_interger]

,num_float,float_decimal,num_interger,den_interger
45,13.5,.5,NaN,10
695,9.75,.75,NaN,10
763,11.27,.27,NaN,10
1712,11.26,.26,NaN,10


In [133]:
# copy 
tweet_clean['rating_numerator']=ratings.num_float.astype(float)
tweet_clean['rating_denominator']=ratings.den_interger.astype(float)

#### Test

In [134]:
tweet_clean.rating_numerator.unique()

array([  1.30000000e+01,   1.20000000e+01,   1.40000000e+01,
         1.35000000e+01,   1.10000000e+01,   1.00000000e+01,
         6.00000000e+00,   1.50000000e+01,   0.00000000e+00,
         8.40000000e+01,   2.40000000e+01,   9.75000000e+00,
         5.00000000e+00,   1.12700000e+01,   3.00000000e+00,
         7.00000000e+00,   8.00000000e+00,   9.00000000e+00,
         4.00000000e+00,   1.65000000e+02,   1.77600000e+03,
         2.04000000e+02,   5.00000000e+01,   9.90000000e+01,
         8.00000000e+01,   4.50000000e+01,   6.00000000e+01,
         4.40000000e+01,   1.43000000e+02,   1.21000000e+02,
         1.12600000e+01,   2.00000000e+00,   1.44000000e+02,
         8.80000000e+01,   1.00000000e+00,   4.20000000e+02])

In [135]:
tweet_clean.rating_denominator.unique()

array([  10.,   70.,    7.,  150.,   11.,  170.,   20.,   50.,   90.,
         80.,   40.,  130.,  110.,  120.,    2.])

In [136]:
tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2006 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2006 non-null int64
in_reply_to_status_id         2006 non-null object
in_reply_to_user_id           2006 non-null object
timestamp                     2006 non-null datetime64[ns]
source                        2006 non-null object
text                          2006 non-null object
retweeted_status_id           2006 non-null object
retweeted_status_user_id      2006 non-null object
retweeted_status_timestamp    13 non-null datetime64[ns]
expanded_urls                 2006 non-null object
rating_numerator              2006 non-null float64
rating_denominator            2006 non-null float64
name                          2006 non-null object
doggo                         2006 non-null object
floofer                       2006 non-null object
pupper                        2006 non-null object
puppo                         2006 non-null object
dtypes: dat

#### Define
- Melt stages (doggo, floofer, pupper, puppo) into single column

#### Clean

In [137]:
tweet_clean = pd.melt(tweet_clean, id_vars=['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp', 'source', 'text',
                              'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp',
                              'expanded_urls', 'rating_numerator', 'rating_denominator', 'name'],
        var_name='stage', value_vars=['doggo', 'floofer', 'pupper', 'puppo'], value_name = 'dog_stage'
       )


In [138]:
# drop the stage column after melting
tweet_clean.drop('stage', axis=1, inplace=True)

In [139]:
# drop duplicates caused due to melting
tweet_clean = tweet_clean.sort_values('dog_stage').drop_duplicates('tweet_id', keep = 'last')

#### Test

In [140]:
tweet_clean.dog_stage.value_counts()

NaN        1697
pupper      215
doggo        63
puppo        23
floofer       8
Name: dog_stage, dtype: int64

In [141]:
tweet_clean.shape

(2006, 14)

#### Define
- Move data from 3 dataframes to one master

#### Clean

In [142]:
# All three should have same number of rows as they are related to one tweet_id
images_clean.shape, tweet_clean.shape, likes_clean.shape

((2006, 12), (2006, 14), (2006, 3))

In [143]:
# merge tweet_clean and images_clean using tweet_id
master_df = tweet_clean.merge(images_clean, on='tweet_id')

In [144]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2006 entries, 0 to 2005
Data columns (total 25 columns):
tweet_id                      2006 non-null int64
in_reply_to_status_id         2006 non-null object
in_reply_to_user_id           2006 non-null object
timestamp                     2006 non-null datetime64[ns]
source                        2006 non-null object
text                          2006 non-null object
retweeted_status_id           2006 non-null object
retweeted_status_user_id      2006 non-null object
retweeted_status_timestamp    13 non-null datetime64[ns]
expanded_urls                 2006 non-null object
rating_numerator              2006 non-null float64
rating_denominator            2006 non-null float64
name                          2006 non-null object
dog_stage                     2006 non-null object
jpg_url                       2006 non-null object
img_num                       2006 non-null int64
p1                            2006 non-null object
p1_conf     

In [145]:
# merge master_df and likes_clean on tweet_id
master_df = master_df.merge(likes_clean, on='tweet_id')

#### Test

In [146]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2006 entries, 0 to 2005
Data columns (total 27 columns):
tweet_id                      2006 non-null int64
in_reply_to_status_id         2006 non-null object
in_reply_to_user_id           2006 non-null object
timestamp                     2006 non-null datetime64[ns]
source                        2006 non-null object
text                          2006 non-null object
retweeted_status_id           2006 non-null object
retweeted_status_user_id      2006 non-null object
retweeted_status_timestamp    13 non-null datetime64[ns]
expanded_urls                 2006 non-null object
rating_numerator              2006 non-null float64
rating_denominator            2006 non-null float64
name                          2006 non-null object
dog_stage                     2006 non-null object
jpg_url                       2006 non-null object
img_num                       2006 non-null int64
p1                            2006 non-null object
p1_conf     

In [147]:
master_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,favorite_count,retweet_count
0,667435689202614272,nan,nan,2015-11-19 20:14:03,"<a href=""http://twitter.com/download/iphone"" r...",Ermergerd 12/10 https://t.co/PQni2sjPsm,nan,nan,NaT,https://twitter.com/dog_rates/status/667435689...,...,0.999091,True,miniature_pinscher,0.000450,True,black-and-tan_coonhound,0.000157,True,308,85
1,667437278097252352,nan,nan,2015-11-19 20:20:22,"<a href=""http://twitter.com/download/iphone"" r...",Never seen this breed before. Very pointy pup....,nan,nan,NaT,https://twitter.com/dog_rates/status/667437278...,...,0.989154,False,bath_towel,0.006300,False,badger,0.000966,False,453,245
2,667443425659232256,nan,nan,2015-11-19 20:44:47,"<a href=""http://twitter.com/download/iphone"" r...",Exotic dog here. Long neck. Weird paws. Obsess...,nan,nan,NaT,https://twitter.com/dog_rates/status/667443425...,...,0.980815,False,drake,0.006918,False,hen,0.005255,False,800,594
3,667453023279554560,nan,nan,2015-11-19 21:22:56,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Meet Cupcake. I would do unspeakable things fo...,nan,nan,NaT,https://twitter.com/dog_rates/status/667453023...,...,0.825670,True,French_bulldog,0.056639,True,Staffordshire_bullterrier,0.054018,True,319,91
4,667455448082227200,nan,nan,2015-11-19 21:32:34,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is Reese and Twips. Reese protects Twips....,nan,nan,NaT,https://twitter.com/dog_rates/status/667455448...,...,0.676376,True,Irish_terrier,0.054933,True,Yorkshire_terrier,0.040576,True,194,60


In [148]:
master_df.shape

(2006, 27)

### Store the cleaned dataset 

In [149]:
# write master_df to .csv file
master_df.to_csv('twitter_archive_master.csv', index=False)